# **Tracking Capacity Unit (CU) Consumption for a DAX Query**

# Check out this [blog post](https://bits2bi.com/2025/10/05/tracking-capacity-unit-consumption-for-dax-queries-with-fabric-notebooks/) for details about this notebook

# Prepare the environment

In [ ]:
# Install the latest version of semantic link

## ........ UNCOMMENT THE FOLLOWING LINE IF YOU'RE RUNNING SPARK 3.3 OR BELOW OR TO UPDATE THE SEMANTIC LINK TO THE LATEST VERSION ........ ##

#%pip install -U semantic-link

In [ ]:
# Import the Semantic Link Python library
import sempy.fabric as fabric

# Import other libraries and modules
from datetime import datetime, timedelta
import time

In [ ]:
## ........ ENSURE THE USER NAME DISPLAYED MATCHES WITH YOUR UPN IF NOT THEN MANUALLY SET THE UPN IN THE dax_query_user VARIABLE ........ ##

# Get the name of the user running the notebook
# This user name will be used to filter the operations in the DAX query to the Capacity Metric semantic model
dax_query_user = mssparkutils.env.getUserName()

print(dax_query_user)

# Execute the DAX query

In [ ]:
# Declare and assign variables

## ........ PROVIDE THE NAME OR ID OF YOUR TARGET WORKSPACE, TARGET SEMANTIC MODEL, THE CAPACITY ID OF THE TARGET WORKSPACE, AND THE DAX QUERY IN FOLLOWING VARIABLES ........ ##

target_workspace = "<TARGET_WORKSPACE_NAME_OR_ID>"
target_semantic_model = "<TARGET_SEMANTIC_MODEL_NAME_OR_ID>"
target_capacity_Id = "<CAPACITY_ID_OF_TARGET_WORKSPACE>"

# DAX query to run against the target semantic model

target_dax_query = f"""
<DAX_QUERY_TO_RUN>
"""

target_workspace = "1879c29f-aac0-4675-a2e8-7692826250a6"
target_semantic_model = "82a38141-0f3e-41ab-b1f8-36aafa192711"
target_capacity_Id = "D16EFC02-0391-43BD-8841-47A044DD67CE"

target_dax_query = """
DEFINE
	VAR __DS0FilterTable = 
		TREATAS({"6 Months"}, 'Period'[Period])

	VAR __DS0FilterTable2 = 
		TREATAS({"Personal",
			"Regular"}, 'Workspaces'[Workspace Type])

	VAR __DS0FilterTable3 = 
		TREATAS({"Active"}, 'Workspaces'[Workspace State])

	VAR __DS0Core = 
		SUMMARIZECOLUMNS(
			'Calendar'[Year Month],
			'Calendar'[Year Month Number],
			__DS0FilterTable,
			__DS0FilterTable2,
			__DS0FilterTable3,
			"Data_Source_Users_in_Selected_Period", '_Measures'[Data Source Users in Selected Period]
		)

	VAR __DS0PrimaryWindowed = 
		TOPN(1001, __DS0Core, 'Calendar'[Year Month Number], 1, 'Calendar'[Year Month], 1)

EVALUATE
	__DS0PrimaryWindowed

ORDER BY
	'Calendar'[Year Month Number], 'Calendar'[Year Month]"""

In [ ]:
# Execute DAX query against the target semantic model
# https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python#sempy-fabric-evaluate-dax

dax_result = fabric.evaluate_dax(
    workspace = target_workspace, 
    dataset = target_semantic_model,
    dax_string = target_dax_query
)

display(dax_result)

In [ ]:
# The billing of Capacity Units starts from the beginning of the next TimePoint i.e., next 30 seconds mark.
# To automate the TimePoint in the DAX query to the Capacity Metric semantic model:
# 1. Get the current utc time in the dax_query_execution_time variable
# 2. Round up the time to the next 30 second mark
# 3. Format the time to fit the DAX query as DAX DATE() + TIME()
# This time will be used as the TimePoint in the DAX query to the Capacity Metric semantic model.

# 1. Get the current utc time in the dax_query_execution_time variable
dax_query_execution_time = datetime.utcnow()

# 2. Round up the time to the next 30 second mark
rounded_seconds = 30 if dax_query_execution_time.second < 30 else 0
extra_minutes = 1 if dax_query_execution_time.second >= 30 else 0

rounded_time = dax_query_execution_time.replace(
    second=rounded_seconds,
    microsecond=0
) + timedelta(minutes=extra_minutes)


# 3. Format the time to fit the DAX query as DAX DATE() + TIME()
operation_TimePoint = f"DATE({rounded_time.year}, {rounded_time.month}, {rounded_time.day}) + TIME({rounded_time.hour}, {rounded_time.minute}, {rounded_time.second})"

## ........ ALTERNATIVELY, SET THE TIMEPOINT MANUALLY ........ ##
# operation_TimePoint = "DATE(YYYY, MM, DD) + TIME(HH, MM, SS)"

#print(operation_TimePoint)

# Track the Capacity Consumption

In [ ]:
# The details of an operation appears in the capacity metrics semantic model in approx 6 minutes
# The waitTime variable is used to set the sleep timer before running the DAX query to the Capacity Metric semantic model

# Set wait time in seconds
waitTime = 6 * 60  # 6 minutes

In [ ]:
## ........ PROVIDE THE NAME OR ID OF YOUR FABRIC CAPACTIY METRICS WORKSPACE AND FABRIC CAPACTIY METRICS SEMANTIC MODEL ........ ##

capacity_metric_workspace = "<CAPACITY_METRIC_WORKSPACE_NAME_OR_ID>"
capacity_metric_semantic_model = "<CAPACITY_METRIC_SEMANTIC_MODEL_NAME_OR_ID>"

capacity_metric_workspace = "212091bd-0616-4370-91e8-c2b2fcf48b25"
capacity_metric_semantic_model = "dc9b689e-fb5a-4261-8e09-ffbb80091ce9"

In [ ]:
# DAX query to extract capacity consumption data from the capacity metrics semantic model

capacity_consumption_dax_query = f"""
// DAX query to return the capacity consumption by the interactive operation for a capacity during a TimePoint
DEFINE
    // Change the TimePoint MPARAMETER to get details of a desired TimePoint
    MPARAMETER 'TimePoint' =
        {operation_TimePoint}
    // Provide a capacity ID
    MPARAMETER 'CapacitiesList' = {{ "{target_capacity_Id}" }}
    VAR __WorkspaceId =
        TREATAS ( {{ "{target_workspace}" }}, 'Items'[Workspace Id] )
    VAR __SemanticModelId =
        TREATAS ( {{ "{target_semantic_model}" }}, 'Items'[Item Id] )
    VAR __User =
        TREATAS ( {{ "{dax_query_user}" }}, 'Timepoint Interactive Detail'[User] )
    VAR __TimePointInteractiveOperations =
        SUMMARIZECOLUMNS (
            'Timepoint Interactive Detail'[Operation start time],
            'Timepoint Interactive Detail'[Operation end time],
            'Timepoint Interactive Detail'[Status],
            'Timepoint Interactive Detail'[Operation],
            'Timepoint Interactive Detail'[User],
            'Items'[Workspace Id],
            'Items'[Workspace name],
            'Items'[Item kind],
            'Items'[Item Id],
            'Items'[Item name],
            __WorkspaceId,
            __SemanticModelId,
            __User,
            "Operation Duration (s)", CALCULATE ( SUM ( 'Timepoint Interactive Detail'[Duration (s)] ) ),
            "TimePoint CU", CALCULATE ( SUM ( 'Timepoint Interactive Detail'[Timepoint CU (s)] ) ),
            "Total CU", CALCULATE ( SUM ( 'Timepoint Interactive Detail'[Total CU (s)] ) )
        )

EVALUATE
__TimePointInteractiveOperations
ORDER BY
    'Timepoint Interactive Detail'[Operation start time] DESC,
    'Timepoint Interactive Detail'[Operation end time]
"""

#print(capacity_consumption_dax_query)

In [ ]:
# Wait at least 6 minutes before running the DAX query to the Capacity Metric semantic model to ensure operation details are available.

print(f"Waiting for {waitTime} seconds before running the DAX query to the Capacity Metric semantic model to ensure operation details are available.")
time.sleep(waitTime)

# Execute DAX query to extract capacity consumption data from the capacity metrics semantic model
# https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python#sempy-fabric-evaluate-dax

print("Getting CU consumption details!!!")

capacity_consumption_result = fabric.evaluate_dax(
    workspace = capacity_metric_workspace, 
    dataset = capacity_metric_semantic_model,
    dax_string = capacity_consumption_dax_query
)

display(capacity_consumption_result)